# Executable in ADS
Detects the creation of an ADS data stream that contains an executable (non-empty imphash)

## Rule Content
```
- title: Executable in ADS
  id: b69888d4-380c-45ce-9cf9-d9ce46e67821
  status: experimental
  description: Detects the creation of an ADS data stream that contains an executable
    (non-empty imphash)
  references:
  - https://twitter.com/0xrawsec/status/1002478725605273600?s=21
  tags:
  - attack.defense_evasion
  - attack.t1027
  - attack.s0139
  author: Florian Roth, @0xrawsec
  date: 2018/06/03
  logsource:
    product: windows
    service: sysmon
    definition: 'Requirements: Sysmon config with Imphash logging activated'
    category: null
  detection:
    selection:
      EventID: 15
    filter:
      Imphash: '00000000000000000000000000000000'
    condition: selection and not filter
  fields:
  - TargetFilename
  - Image
  falsepositives:
  - unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"15" AND (NOT (hash_imphash:"00000000000000000000000000000000")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()